# Bag-of-WordsとLightGBMで恋愛コラムの本文からライターを判定する

## 概要

- [Googirl](https://googirl.jp/)という女性向け恋愛コラムのWebサイトがあります。
- このサイトは、コラムの記事ごとにライターが異なります。コラムの本文からそのコラムを書いたライターを分類するLightGBMモデルを作ってみました。特徴量はBag-of-Words (BoW) です。
  - コラムの総数が多い上位10人のライターに絞っています。
    - つまり、記事のBoWを特徴量にした5クラス分類タスクです。
  - なお、記事数は7000件程度、1本の記事は1000文字程度です。
  - LightGBMはoptuna.integration.lightgbmを使いました。
- 予測精度は92%程度でした。

## データ

- 対象は、Googirlの[恋愛・結婚カテゴリ](https://googirl.jp/renai/)のうち、2017/1/1～2021/6/15に書かれたコラムです。そのうち、投稿数が多い順に10人のライターのコラムのみを取り出しました。
  - この10人で、全てのコラムの7割程度を占めます。
  - 他にもカテゴリはありますが、同じカテゴリの文章を分類する方が難易度が高く面白いと思ったので、恋愛・結婚カテゴリのみに絞りました。
  - ただし、続き物のコラムは最初の1本のみを取り出しました。
    - vol.1～vol.9や第1位～第9位、前編・後編のように、同じライターが同じテーマを複数の記事に分割していることが多いです。この場合、vol.1、第1位、前編のみをデータに残し、それ以外の記事は含めていないということです。というのも、これらの記事は同じテーマであるため、ある意味ライターを予測できるのが当たり前だからです。
  - 全部で6955件の記事です。
- Googirlの恋愛・結婚カテゴリをスクレイピングし、タイトル、本文、ライター名を列に持つpd.DataFrameを事前に用意しています。
  - 十分なスリープを挟みました。


In [25]:
import MeCab
import neologdn
import pandas as pd
import numpy as np
from datetime import datetime
import optuna.integration.lightgbm as lgb
from optuna.logging import set_verbosity
from lightgbm import Dataset, early_stopping, log_evaluation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from tqdm import tqdm
import time
from customized_tagger import CustomizedTagger

# 読み込みと前処理 ---------------------------------------------------------------------
freq_author=10
googirl_article=pd.read_csv("../data/googirl/googirl_article_renai_and_marriage_20210615.csv")
googirl_article=(
    googirl_article
    .assign(date=lambda d: pd.to_datetime(d.date))
    .loc[lambda d: d.date>=datetime(2017,1,1)]
    .assign(
        title=lambda d: d.title.map(neologdn.normalize),
    ).loc[
        # [vV]ol. 1や前編、第1位以外の連番のタイトルの記事を削除する
        (lambda d: 
            ~d.title.str.contains(r"[vV]ol\. ?[2-9]") &
            ~d.title.str.contains("[中後]編") &
            ~d.title.str.contains("第[2-9]位")
        )
    ]
)

# ライター別の記事数
n_by_author=(
    googirl_article.author
    .value_counts(ascending=False)
    # indexにあるauthorを列として戻すため
    .reset_index()
    .rename(columns={"index":"author","author":"n"})
)
print(n_by_author.head(freq_author).to_markdown())
used_authors=n_by_author.author.head(freq_author).to_list()
print(used_authors)

|    | author       |    n |
|---:|:-------------|-----:|
|  0 | 松はるな     | 1544 |
|  1 | 和           |  729 |
|  2 | 小林ユリ     |  682 |
|  3 | Waxy         |  677 |
|  4 | 美佳         |  667 |
|  5 | 占い師シータ |  665 |
|  6 | きいろ       |  587 |
|  7 | チオリーヌ   |  524 |
|  8 | 小林リズム   |  497 |
|  9 | 原桃子       |  383 |
['松はるな', '和', '小林ユリ', 'Waxy', '美佳', '占い師シータ', 'きいろ', 'チオリーヌ', '小林リズム', '原桃子']


## 前処理

- 以下を削除しました。
  - htmlタグ
  - 改行
  - 下三角形やかぎかっこの前の中黒
    - ディバイダー的に使われているものです。
  - インタビューした人の年齢など
    - 削除しないとBoWを作る上でノイズになりえます。
- 正規化はneologd.normalizeを用いました。
  - NFKC正規化にいくつかの表記の揺れを直してくれる便利なライブラリです。

In [26]:
article=(
    googirl_article
    .loc[lambda d: d.author.isin(used_authors)]
    # urlの最後のサブディレクトリ（https://hoge.com/fuga/piyo/のpiyo）を取り出して一意なキーとして使う
    .assign(id_url=lambda d: d.url.str.extract("([^/]+)(?=/$)",expand=True))
    # 連番を振る
    .assign(id=lambda d: pd.RangeIndex(start=0,stop=len(d),step=1).astype(str))
    .assign(
        text=lambda d: d.text.str.replace(r"[\(（].*?[0-9]+[歳才代].*?[）\)]","",regex=True)
                                .str.replace(r"\n","",regex=True)
                                .str.replace(r"\[\[\[p\]\]\]","",regex=True)
                                .str.replace("▽","",regex=True)
                                .str.replace("・(?= ?「)","",regex=True)
                                .map(neologdn.normalize)
     )
     .reset_index()
     .filter(["id","id_url","url","date","title","author","text"])
)

df=article.filter(["id","id_url","author","title","text"])
df.head(5)

,id,id_url,author,title,text
0,0,210615020,きいろ,こういう言葉がほしかった!もらうとうれしい恋愛アドバイス4選,恋のアドバイスといっても、どんな言葉をかけたらいいのか悩みますよね。「もっといい人がいるよ!...
1,1,210614016,原桃子,性格悪すぎてひどい…!圧倒的にモテない女子の特徴TOP7【第1位】,可愛くてオシャレなのに、なぜかいつまでたっても彼氏ができない女子っていますよね。そんな女子た...
2,2,210614007,小林リズム,"「もっといい人がいるはず」と別れを決意した女性の“その後""とは?",結婚に向いていそうな人と付き合ったものの、心のどこかで納得できておらず……。「もっといい人が...
3,3,210614009,美佳,12星座別!彼の究極の欠点とは?【前編】,誰にでも一つくらいは欠点があるもの。付き合う上では、相手の欠点を受け入れられるかどうかがかな...
4,4,210614006,松はるな,何が違うの?「彼女にしたい女」と「友達止まりの女」の3つの差,いい感じに仲よくなってきたな……と思いきや、「俺たちっていい友達だよな!」とまさかの友達発言...


## 形態素解析と特徴量のBoW作成

- 環境
  - Windows 10 + i9-9900K
  - Python 3.10 + MeCab 32bit (v0.996) + [mecab](https://pypi.org/project/mecab/) (v.0.996.3) 
    - 辞書はデフォルトのIPA辞書と2020/8/20（本記事を執筆時で最新）のmecab-ipadic-NEologdです。
    - MeCabのPythonバインディングはmecabを用いています。
      - 私の環境ではmecab-python3のインストールに失敗したためです。
- 特徴量
  - 名詞のみ用いています。
  - ストップワードに[Slothlib](http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt)を用いています。
  - BoWは2値にしています。
  - scikit-learnのCountVectorizerのmin_dfを0.001にしています。
    - つまり、6955記事の中の6記事以下でしか出現しない単語を削除しています。
    - min_dfを0にすると17255個の特徴量（＝17255次元）ですが、0.001にしたことで5772個まで減りました。
- CountVectorizerには、原型のまま分かち書きをして名詞のみ残したものを渡しています。
  - 実装のコツとして、MeCab.Tagger("-Ochasen");MeCab.Tagger.parseToNodeの結果をpd.DataFrameで返すラッパーの自作クラスを作っています（末尾のセルを参照）。このDataFrameから品詞が名詞のもののみを抽出して、その原型の空白を挟んでくっつけています。
- MeCab.Tagger()は、MeCab.Tagger("-r .mecabrcのパス")で使用する.mecabrcを指定できますので、UTF-8版の辞書を記載した.mecabrcを引数に与えることでUTF-8の辞書を使えるようにしています。
  - 私はRとPythonの両方でMeCabを用いています。Windows環境でのMeCabは、RではShift-JIS、PythonではUTF-8の辞書を用いるため、デフォルトのipadicとNEologdともにShift-JISとUTF-8の両方の辞書を作成しています。Shift-JISの辞書とUTF-8の辞書それぞれを記載した.mecabrcを作っておくことで、RとPythonでMeCabを使い分けることができます。

In [27]:
# 形態素解析と特徴量を作る ---------------------------------------------------------------------
ct=CustomizedTagger("-Ochasen -r ../utf8.mecabrc")
texts=df.text.to_list()
# 形態素解析し、名詞のみを取り出して空白区切りの分かち書きにする
# [np.array(["吾輩 猫"],np.array(["名前"])]のようなもの
wakati=[np.array(ct.wakati_base_extract(j,["名詞"])) for i,j in enumerate(tqdm(texts))]
# np.array(["吾輩 猫","名前"])にする
wakati=np.concatenate(wakati)

# Bag-of-Wordを作る
stop_words=(
    pd.read_csv("http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt",header=None)
    .set_axis(["word"],axis=1)
    .word
    .to_list()
)
cv=CountVectorizer(min_df=0.001,binary=True,stop_words=stop_words)
cv_transform=cv.fit_transform(wakati)
df_x=pd.DataFrame(cv_transform.toarray(),columns=cv.get_feature_names_out())
x=np.array(df_x)

# 目的変数のauthor列をラベルエンコーティングする
le=LabelEncoder()
df_y=df.assign(target=lambda d: le.fit_transform(d.author))
y=np.array(df_y.target)

100%|██████████| 6955/6955 [00:17<00:00, 403.69it/s]


In [28]:
df_x.iloc[:,0:10]

,100,100倍,100年,100点,10万円,10人,10代,10分,10年,10歳
0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
6950,0,0,0,0,0,0,0,0,0,0
6951,0,0,0,0,0,0,0,0,0,0
6952,0,0,0,0,0,0,0,0,0,0
6953,0,0,0,0,0,0,0,0,0,0


In [29]:
df_y.target

0       1
1       4
2       7
3       9
4       8
       ..
6950    1
6951    1
6952    2
6953    8
6954    6
Name: target, Length: 6955, dtype: int32

## 学習と評価

- 環境
  - scikit-learn: v1.0.2
  - lightgbm: v3.3.2
  - optuna: v2.10.0
- LightGBMのライブラリはoptuna.integration.lightgbmを用いました。optunaのベイズ最適化でハイパーパラメータ探索を行いました。
- 交差検証はNested CVです。
  - 外側のCV（学習と予測のFold）と内側のCV（学習の中のハイパーパラメータの調整のFold）のいずれも、5-FoldのStratified Foldとしています。
    - Stratifiedなのはラベルが不均衡だからです。
- i9-9900Kの私の環境で3～4時間で終了したと思います。メモリは3GB程度使っていました。

In [ ]:
# 学習と予測 ---------------------------------------------------------------------
params = {
    'objective': 'multiclass',
    'num_class': freq_author,
    'metric': 'multi_logloss',
    'boosting_type': 'gbdt',
    'deterministic': True,
    'force_row_wise': True
    }
set_verbosity(1)
callbacks=[
    early_stopping(stopping_rounds=20),
    log_evaluation(period=20)
]

start_time=time.time()
pred_all=[]
true_all=[]
test_idx_all=[]
# 外側のCV（学習データと評価データ）
fold=StratifiedKFold(n_splits=5,shuffle=True,random_state=1234)
for train_idx, test_idx in fold.split(x,y):
    x_train=x[train_idx]
    y_train=y[train_idx]
    x_test=x[test_idx]
    y_test=y[test_idx]

    lgb_train=lgb.Dataset(x_train,y_train)
    lgb_test=lgb.Dataset(x_test,y_test)
    # 内側のCV（学習データのうち、ハイパーパラメータの学習データと評価データに分ける）
    inner_fold=StratifiedKFold(n_splits=5,shuffle=True,random_state=1234)
    tuner_cv = lgb.LightGBMTunerCV(
        params,
        lgb_train,
        num_boost_round=1000,
        folds=inner_fold,
        callbacks=callbacks,
        optuna_seed=1234,
        return_cvbooster=True
    )
    tuner_cv.run()
    # 最もよいモデルを取り出し、保存してからそのモデルをロードし、それでテストデータを予測する
    best_model=tuner_cv.get_best_booster()
    best_model.save_model("lgb_model.txt")
    best_model=lgb.Booster(model_file="lgb_model.txt")
    pred=best_model.predict(x_test)
    pred=np.argmax(pred, axis=1)
    
    pred_all.append(pred)
    true_all.append(y_test)
    test_idx_all.append(test_idx)
    print(confusion_matrix(pred,y_test))
    print(accuracy_score(pred,y_test))

end_time=time.time()

pred_all_concat=np.concatenate(pred_all)
true_all_concat=np.concatenate(true_all)
test_idx_all_concat=np.concatenate(test_idx_all)
df_pred=pd.DataFrame(zip(test_idx_all_concat,pred_all_concat),columns=["id","pred"]).sort_values("id").assign(id=lambda d: d.id.astype(str))

# 予測結果を結合したdf
df_res=(
    pd.merge(df,df_pred,on="id",how="inner")
    .assign(pred=lambda d: le.inverse_transform(d.pred))
)
df_res.to_csv("../googirl_classify_result.csv")

## 予測結果

混同行列は行が正解のラベル、列が予測値のラベルです。

In [ ]:
# 混同行列と予測精度を学習したモデルから求めるならこれを使うが、
# 本notebookでは事前に回したコードで求めている予測結果を読み込むためコメントアウト
# conf_mat=pd.DataFrame(
#     confusion_matrix(true_all_concat,pred_all_concat),
#     # column,indexともに0-9なので
#     columns=le.inverse_transform(np.unique(y)),
#     index=le.inverse_transform(np.unique(y)),
# )
# print(conf_mat)
# print(classification_report(df_res.author,df_res.pred,digits=5))
# print("accuracy_score: " + str(accuracy_score(pred_all_concat,true_all_concat)))

In [30]:
df_res=pd.read_csv("../googirl_classify_result.csv")
# 行がtrue, 列がpred
conf_mat=pd.DataFrame(
    confusion_matrix(df_res.author,df_res.pred),
    # column,indexともに0-9なので
    columns=le.inverse_transform(np.unique(y)),
    index=le.inverse_transform(np.unique(y)),
)
print(classification_report(df_res.author,df_res.pred,digits=5))
print("accuracy_score: " + str(accuracy_score(df_res.author,df_res.pred)))

              precision    recall  f1-score   support

        Waxy    0.95067   0.93944   0.94502       677
         きいろ    0.83956   0.91823   0.87714       587
       チオリーヌ    0.93933   0.91603   0.92754       524
      占い師シータ    0.95542   0.96692   0.96114       665
         原桃子    0.89118   0.79112   0.83817       383
           和    0.93046   0.91770   0.92403       729
        小林ユリ    0.92377   0.90616   0.91488       682
       小林リズム    0.90020   0.88934   0.89474       497
        松はるな    0.94997   0.97150   0.96061      1544
          美佳    0.94260   0.93553   0.93905       667

    accuracy                        0.92797      6955
   macro avg    0.92232   0.91520   0.91823      6955
weighted avg    0.92832   0.92797   0.92777      6955

accuracy_score: 0.9279654924514738


In [31]:
conf_mat

,Waxy,きいろ,チオリーヌ,占い師シータ,原桃子,和,小林ユリ,小林リズム,松はるな,美佳
Waxy,636,8,4,4,0,3,14,2,1,5
きいろ,4,539,5,4,3,7,8,4,10,3
チオリーヌ,4,9,480,1,5,3,2,16,1,3
占い師シータ,1,8,1,643,0,2,4,4,2,0
原桃子,7,13,5,2,303,14,4,7,18,10
和,2,12,0,5,9,669,4,2,14,12
小林ユリ,6,16,3,4,8,5,618,6,15,1
小林リズム,3,15,9,6,2,5,6,442,8,1
松はるな,4,13,2,3,3,5,5,6,1500,3
美佳,2,9,2,1,7,6,4,2,10,624


In [32]:
# 分類結果
df_res.filter(["id","id_url","author","title","text","pred"]).head(5)

,id,id_url,author,title,text,pred
0,0,210615020,きいろ,こういう言葉がほしかった!もらうとうれしい恋愛アドバイス4選,恋のアドバイスといっても、どんな言葉をかけたらいいのか悩みますよね。「もっといい人がいるよ!...,きいろ
1,1,210614016,原桃子,性格悪すぎてひどい…!圧倒的にモテない女子の特徴TOP7【第1位】,可愛くてオシャレなのに、なぜかいつまでたっても彼氏ができない女子っていますよね。そんな女子た...,原桃子
2,2,210614007,小林リズム,"「もっといい人がいるはず」と別れを決意した女性の“その後""とは?",結婚に向いていそうな人と付き合ったものの、心のどこかで納得できておらず……。「もっといい人が...,小林リズム
3,3,210614009,美佳,12星座別!彼の究極の欠点とは?【前編】,誰にでも一つくらいは欠点があるもの。付き合う上では、相手の欠点を受け入れられるかどうかがかな...,美佳
4,4,210614006,松はるな,何が違うの?「彼女にしたい女」と「友達止まりの女」の3つの差,いい感じに仲よくなってきたな……と思いきや、「俺たちっていい友達だよな!」とまさかの友達発言...,松はるな


## 感想

- 同じカテゴリの文章なので分類が難しいかと思いましたが、思いの外高い精度が出ました。
  - ライターによって使っている単語が若干違うからだと思います。
    - ライターごとにTF-IDFの高い単語を並べてみると、ライターによって使う単語の特徴が出ます。
  - それでも人間が読んでライターを当てるのは難しいです。
- BoWは古典的な手法ですが、このようにタスクによっては強力です。
- 改行を削除するなど、前処理を丁寧に行うと精度が上がりました。
- コラムに目を通してこのデータセットに適した前処理を考える所と、想定通り前処理できているか確認して都度正規表現を書く所が一番大変でした。

In [33]:
# ライター別にTF-IDFの上位100単語を並べる

text_by_author=[" ".join(df.loc[lambda d: d.author==i].text) for i in used_authors]
wakati_by_author=np.concatenate(
    [np.array(ct.wakati_base_extract(j,["名詞"])) for i,j in enumerate(tqdm(text_by_author))]
)

tiv=TfidfVectorizer(max_features=1000,min_df=0.05,max_df=0.95,binary=False)
# author x wordsのnp.array
tiv_transform=tiv.fit_transform(wakati_by_author).toarray()

feature_names = tiv.get_feature_names_out()
index = tiv_transform.argsort(axis=1)[:,::-1]
tfidf=(
    pd.DataFrame([feature_names[x[:100]] for x in index])
    .transpose()
    .reset_index()
    .set_axis(["rank"]+used_authors,axis=1)
    .assign(rank=lambda d: (d["rank"]+1).astype(str) + "位")
)

100%|██████████| 10/10 [00:17<00:00,  1.73s/it]


## （参考）MeCabの出力のパースをする自作クラス

In [ ]:
import MeCab
import pandas as pd

class CustomizedTagger(MeCab.Tagger):
    cols=["surface","pos","pos1","pos2","pos3","form1","form2","base","yomi","hatsuon"]

    # MeCab.Tagger.parseToNodeをpd.DataFrameで返す
    def parse(self,text: str) -> pd.DataFrame:
        res=[]
        node=self.parseToNode(text)
        while node:
            surface=[node.surface]
            feature=node.feature.split(",")
            if (feature[0]!="BOS/EOS"):
                res.append([*surface,*feature])
            node=node.next
        res_df=pd.DataFrame(res,columns=self.cols)
        return res_df

    # 原型で分かち書きし、posに指定した品詞のみ取り出す
    def wakati_base_extract(self,text: str,pos: list[str]) -> list[str]:
        parsed=self.parse(text)
        extracted_list=parsed.loc[lambda d: d.pos.isin(pos)].base.to_list()
        res=" ".join(extracted_list)
        res=[res]
        return res